In [ ]:
#  Copyright (c) 2017-2025 KAPPA
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
# This file is part of Orchid and related technologies.
#

# Example: Searching for project data frames

This notebook illustrates using the Orchid* Python API and the `pandas` package to
search for and use project data frames.

(*Orchid is a mark of KAPPA)

## 0.5 Import packages

The only import needed for the Orchid Python API is `orchid` itself.

In [ ]:
import orchid

The remaining imports are standard python packages to support the analysis.

In [ ]:
import pprint
import uuid

import matplotlib.pyplot as plt
import pandas as pd
# The following import is included for its "side-effects" of an improved color schemes and
# plot styles. (See the "Tip" in section 9.2 of "Python for Data Analysis" for details.)
import seaborn as sns

## 1.0 Load the .ifrac project

The following code simply captures the configured location of the Orchid training data. It is not needed to
use the Orchid Python API itself, but it is used in this example to load well-known data.

In [ ]:
orchid_training_data_path = orchid.training_data_path()

In [ ]:
project = orchid.load_project(str(orchid_training_data_path.joinpath(
    'Project-frankNstein_Permian_UTM13FT_DF_PR2298_vs263.ifrac')))

## 2.0 Search for a data frame using its object ID

I have copied to the object ID from Orchid itself.

In [ ]:
project_data_frame = project.data_frames().find_by_object_id(
    uuid.UUID('c08e6988-d8f5-4d7b-bccd-de968a5b398b'))
to_display = ('No such data frame' if project_data_frame is None
              else project_data_frame.pandas_data_frame())
to_display

## 3.0 Search for a data frame by its name

In [ ]:
fdi_data_frames = list(project.data_frames().find_by_name('FDI Observations'))
assert(len(fdi_data_frames) == 1)
fdi_data_frame = fdi_data_frames[0]
fdi_data_frame.pandas_data_frame()

## 4.0 Search for a data frame by its display name

In [ ]:
microseismic_data_frames = list(project.data_frames().find_by_display_name(
    'Microseismic Data Frame 01 (Potentially Corrupted)'))
assert(len(microseismic_data_frames) == 1)
microseismic_data_frame = microseismic_data_frames[0]
microseismic_data_frame.pandas_data_frame()

## 5.0 Query for all identifying information

In [ ]:
print('All data frame object IDs')
pprint.pprint(list(project.data_frames().all_object_ids()))

In [ ]:
print('All data frame names')
pprint.pprint(list(project.data_frames().all_names()))

In [ ]:
print('All data frame display names')
pprint.pprint(list(project.data_frames().all_display_names()))

## 7.0 Leverage the full power of pandas

In [ ]:
stage_data_frame = project.data_frames().find_by_object_id(
    uuid.UUID('5304d2ac-dbf8-44db-8dd8-c2203714c456'))
stage_data_frame.pandas_data_frame()

Extract a specific column (the cumulative slurry of a stage)

In [ ]:
stage_data_frame.pandas_data_frame()[' cum slurry [bll]']

Find the maximum value (the volume to pick from the FDI data frame)

In [ ]:
max(fdi_data_frame.pandas_data_frame()['VolumeToPick'])

### 7.1 Extract completion data

In [ ]:
raw_completion_column_names = ['Timestamp', 'DeltaT', 'DeltaP', 'VolumeToPick', 'ProppantMass', 'Energy']

# Extract completion columns into new data frame and change index to time
raw_completion_details = fdi_data_frame.pandas_data_frame()[
    raw_completion_column_names].set_index('Timestamp')

raw_completion_details.describe()

### 7.2 Scale completion data to fit on single plot

In [ ]:
def scale_down_by_1000(v):
    return v / 1000.0

copy_completion_details = raw_completion_details.copy()
copy_completion_details['ProppantMass'] = copy_completion_details['ProppantMass'].transform(scale_down_by_1000)
copy_completion_details['Energy'] = copy_completion_details['Energy'].transform(scale_down_by_1000)
copy_completion_details['VolumeToPick'] = copy_completion_details['VolumeToPick'].transform(scale_down_by_1000)

completion_details = copy_completion_details.rename({'ProppantMass': 'ProppantMass (1000 lbs)',
                                                     'Energy': 'Energy (1000 ft-lb)',
                                                     'VolumeToPick': 'VolumeToPick (Mbbl)'},
                                                    axis='columns', errors='raise')
completion_details

### 7.3 Plot the completion data

In [1]:
_axes = completion_details.plot(figsize=(13, 8), title='Completion Data')

NameError: name 'completion_details' is not defined